In [ ]:
# NLP Lecture @ Strive School - 21st July 2021
# NER update

'''
Since today we are exploring the world of natural language processing, we’ll deepen in the Named Entity Recognition technique: this is just one of the mechanisms that NLP embodies. The recognition of named entities as the process of automatic identification of the entities present in a text and consequent classification into predefined categories such as "person", "organization", "position" is a quite common activity and expect for English, trained models with spaCy offer few labels that could be improved through training.

Following the case study of this morning, try to emulate it in order to label all the brands present in the provided datasets, choosing the one you prefer OR trying to label all them and to train the model to recognize new different entities. The result should be twofold: the final model should be able to recognize brands that it has already seen, but already new ones.
The brands proposed in the dataset concern fashion, cars and food.
In order to test the accuracy of the model, test it with sentences and brands the model has never seen.

Sample of the dataset
---------------------
- Cate Blanchett in Armani Privé. Rating: 8. Concludes as a rare butterfly, or from Rorschach's Test, or from computerized axial tomography.
- I liked everything, recommend it! Another quality Xiaomi product...
- What is the price of that Fiat 500XL?

Info:
- Feel free to change or arrange a new dataset
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)

'''

In [2]:
# STEP 0 - PRE REQUISITES

# python -m spacy download en_core_web_lg

# TBD: Import libraries
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TBD: Load preferred model

with open("food.txt") as file:
    dataset = file.read()

# TBD: Load the dataset and test it as-is

In [4]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(dataset)
print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])

Entities: [('Italian', 'NORP'), ('first', 'ORDINAL'), ('Arrosticini', 'PERSON'), ('Alfredo', 'PERSON'), ('Naples', 'GPE'), ('Zaza', 'PERSON'), ('ApplePie', 'ORG'), ('Bologna', 'GPE'), ('Fiorentina Steak', 'PERSON'), ('Pineapple', 'ORG'), ('First', 'ORDINAL'), ('Bronte', 'PERSON'), ('Coca-cola', 'ORG'), ('Fanta', 'ORG'), ('Pepsi', 'ORG'), ('One', 'CARDINAL'), ('Sorrento', 'GPE'), ('Coffee\nBread', 'ORG'), ('Love', 'WORK_OF_ART'), ('Fatte', 'PERSON'), ('vedrai che il mondo poi ti', 'PERSON'), ('Pastiera', 'PERSON'), ('the United States', 'GPE'), ('Two', 'CARDINAL'), ('two', 'CARDINAL'), ('24 hours', 'TIME'), ('24', 'CARDINAL'), ('two hours', 'TIME')]


In [ ]:
# STEP 1 - TRAIN DATA

# Prepare training data

# TBD: define all the entities by extracting the words and their indexes from the dataset
# expected format is the following:  ("sentence", {"entities": [0,10, "FOOD"]})

In [6]:
words = ["ketchup", "pasta", "carrot", "pizza",
         "garlic", "tomato sauce", "basil", "carbonara",
         "eggs", "cheek fat", "pancakes", "parmigiana", "eggplant",
         "fettucine", "heavy cream", "polenta", "risotto", "espresso",
         "arrosticini", "spaghetti", "fiorentina steak", "pecorino",
         "maccherone", "nutella", "amaro", "pistachio", "coca-cola",
         "wine", "pastiera", "watermelon", "cappuccino", "ice cream",
         "soup", "lemon", "chocolate", "pineapple"]
train_data = []

In [7]:
with open("food.txt") as file:
    dataset = file.readlines()
    for sentence in dataset:
        print("######")
        print("sentence: ", sentence)
        print("######")
        sentence = sentence.lower()
        entities = []
        for word in words:
            word = word.lower()
            if word in sentence:
                start_index = sentence.index(word)
                end_index = len(word) + start_index
                print("word: ", word)
                print("----------------")
                print("start index:", start_index)
                print("end index:", end_index)
                pos = (start_index, end_index, "FOOD")
                entities.append(pos)
        element = (sentence.rstrip('\n'), {"entities": entities})

        train_data.append(element)
        print('----------------')
        print("element:", element)

        ("this is my sentence", {"entities": [0, 4, "PREP"]})
        ("this is my sentence", {"entities": [6, 8, "VERB"]})

######
sentence:  Give me carrot cake.

######
word:  carrot
----------------
start index: 8
end index: 14
----------------
element: ('give me carrot cake.', {'entities': [(8, 14, 'FOOD')]})
######
sentence:  I love simple pizza margherita with tomato sauce and mozzarella.

######
word:  pizza
----------------
start index: 14
end index: 19
word:  tomato sauce
----------------
start index: 36
end index: 48
----------------
element: ('i love simple pizza margherita with tomato sauce and mozzarella.', {'entities': [(14, 19, 'FOOD'), (36, 48, 'FOOD')]})
######
sentence:  I don't like pizza with garlic!

######
word:  pizza
----------------
start index: 13
end index: 18
word:  garlic
----------------
start index: 24
end index: 30
----------------
element: ("i don't like pizza with garlic!", {'entities': [(13, 18, 'FOOD'), (24, 30, 'FOOD')]})
######
sentence:  I like pasta with homemade tomato sauce and basil.

######
word:  pasta
----------------
start index: 7
end index: 12
word:  tomato s

In [8]:
# STEP 2 - UPDATE MODEL
# TBD: load the needed pipeline
# TBD: define the annotations
# TBD: train the model

In [9]:
ner = nlp.get_pipe("ner")

for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [11]:
# Train model
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Train model
with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(30):
        print("Iteration #", iteration)
        # Data shuffle for each iteration
        random.shuffle(train_data)
        losses = {}

        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                # Create an Example object
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], losses=losses, drop=0.1)
                # Update the model
        print("Losses:", losses)

Iteration # 0


C:\Users\fb\.conda\envs\nlp\lib\site-packages\spacy\training\iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "sorrento lemons are famous all over the world" with entities "[(9, 14, 'FOOD')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses: {'ner': 114.07104829854737}
Iteration # 1
Losses: {'ner': 34.22523452919537}
Iteration # 2
Losses: {'ner': 12.455815427731025}
Iteration # 3
Losses: {'ner': 7.996881823516186}
Iteration # 4
Losses: {'ner': 22.634198294737566}
Iteration # 5
Losses: {'ner': 7.717364529653797}
Iteration # 6
Losses: {'ner': 4.664613826794731}
Iteration # 7
Losses: {'ner': 2.5465730308934176}
Iteration # 8
Losses: {'ner': 1.3965096584646193}
Iteration # 9
Losses: {'ner': 4.271436516198889}
Iteration # 10
Losses: {'ner': 1.6053695240980133}
Iteration # 11
Losses: {'ner': 1.3187509828075572}
Iteration # 12
Losses: {'ner': 0.002486412270344278}
Iteration # 13
Losses: {'ner': 2.7942551552748967}
Iteration # 14
Losses: {'ner': 1.0148022463371924}
Iteration # 15
Losses: {'ner': 2.4957942050671322}
Iteration # 16
Losses: {'ner': 1.8020358623235724}
Iteration # 17
Losses: {'ner': 0.00780024542185682}
Iteration # 18
Losses: {'ner': 0.00035670208128146325}
Iteration # 19
Losses: {'ner': 0.43798534997688143}
I

In [12]:
output_dir = Path("/ner/")
nlp.to_disk(output_dir)
print("Saved correctly!")

Saved correctly!


In [ ]:
# STEP 3 - TEST THE UPDATED MODEL

# Load updated model
nlp_updated = spacy.load(output_dir)

# TBD: test with a old sentence
print("entities:", [(ent.text, ent.label_) for ent in doc.ents])

# TBD: test with a new sentence and an old brand

# TBD: test with a new sentence and a new brand